tgb - 11/27/2019 - Training the network for data that was vertically-interpolated on the eps grid

# 0) Imports

In [2]:
from cbrain.imports import *
from cbrain.data_generator import *
from cbrain.cam_constants import *
from cbrain.losses import *
from cbrain.utils import limit_mem
from cbrain.layers import *
from cbrain.data_generator import DataGenerator
import tensorflow as tf
import tensorflow.math as tfm
import tensorflow_probability as tfp
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin

TRAINDIR = '/local/Tom.Beucler/SPCAM_PHYS/'
DATADIR = '/project/meteo/w2w/A6/S.Rasp/SP-CAM/fluxbypass_aqua/'
PREFIX = '8col009_01_'
%cd /filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM

# Otherwise tensorflow will use ALL your GPU RAM for no reason
limit_mem()

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

/home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog


W1129 22:59:03.078180 23271543777088 deprecation_wrapper.py:119] From /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog/cbrain/utils.py:145: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W1129 22:59:03.078960 23271543777088 deprecation_wrapper.py:119] From /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog/cbrain/utils.py:148: The name tf.keras.backend.set_session is deprecated. Please use tf.compat.v1.keras.backend.set_session instead.

W1129 22:59:03.080538 23271543777088 deprecation_wrapper.py:119] From /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog/cbrain/utils.py:148: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



/filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM


# 1) Design NN with RH_eps and T_eps as inputs

## 1.1) Rescaling

In [2]:
scale_dict = {
    'PHQEPS': L_V/G,  
    'TPHYSTNDEPS': C_P/G, 
    'FSNT': 1, 
    'FSNS': 1, 
    'FLNT': 1, 
    'FLNS': 1, 
}

For quick test, use equal pressure thickness for all eps layers (equal weighting in eps space)

In [3]:
PS = 1e5
eps_res = 1e2
dP = PS/eps_res

In [4]:
for v in ['PHQEPS','TPHYSTNDEPS']:
    scale_dict[v] *= dP

In [5]:
save_pickle('./nn_config/scale_dicts/127_POG_scaling.pkl', scale_dict)

## 1.2) Build data generator

In [5]:
in_vars = ['RHEPS','TBPEPS','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQEPS','TPHYSTNDEPS','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/131_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/131_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [6]:
valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/131_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/131_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [7]:
X, Y = valid_gen[12]; X.shape, Y.shape

((1024, 204), (1024, 204))

Reduce number of layers to 5 since less data and wider layers

In [8]:
inp = Input(shape=(204,))
densout = Dense(256, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (3):
    densout = Dense(256, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(204, activation='linear')(densout)
out = LeakyReLU(alpha=0.3)(densout)
NNmodel = tf.keras.models.Model(inp,out)

W1128 07:43:50.146628 23269534324544 deprecation.py:506] From /home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
NNmodel.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 204)]             0         
_________________________________________________________________
dense (Dense)                (None, 256)               52480     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0     

In [13]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG131.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [14]:
NNmodel.compile(tf.keras.optimizers.Adam(),loss=mse)

In [15]:
Nep = 20
NNmodel.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/20
3448/3448 [==============================] - 44s 13ms/step - loss: 31.9895 - val_loss: 32.5142
Epoch 2/20
3448/3448 [==============================] - 43s 12ms/step - loss: 31.6992 - val_loss: 32.5529
Epoch 3/20
3448/3448 [==============================] - 43s 13ms/step - loss: 31.4743 - val_loss: 31.9120
Epoch 4/20
3448/3448 [==============================] - 45s 13ms/step - loss: 31.2693 - val_loss: 31.7280
Epoch 5/20
3448/3448 [==============================] - 45s 13ms/step - loss: 31.1017 - val_loss: 31.7779
Epoch 6/20
3448/3448 [==============================] - 47s 14ms/step - loss: 30.9046 - val_loss: 31.7368
Epoch 7/20
3448/3448 [==============================] - 45s 13ms/step - loss: 30.7238 - val_loss: 31.3456
Epoch 8/20
3448/3448 [==============================] - 45s 13ms/step - loss: 30.5716 - val_loss: 32.7376
Epoch 9/20
3448/3448 [==============================] - 48s 14ms/step - loss: 30.4611 - val_loss: 31.3613
Epoch 10/20
3448/3448 [=======================

# 2) Repeat with larger dataset

In [6]:
in_vars = ['RHEPS','TBPEPS','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQEPS','TPHYSTNDEPS','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/132_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/131_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [7]:
valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/132_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/131_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [8]:
X, Y = valid_gen[19]; X.shape, Y.shape

((1024, 204), (1024, 204))

In [9]:
inp2 = Input(shape=(204,))
densout = Dense(256, activation='linear')(inp2)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (5):
    densout = Dense(256, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(204, activation='linear')(densout)
out2 = LeakyReLU(alpha=0.3)(densout)
NNmodel2 = tf.keras.models.Model(inp2,out2)

W1128 20:34:22.045083 22889390782272 deprecation.py:506] From /home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
NNmodel2.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 204)]             0         
_________________________________________________________________
dense (Dense)                (None, 256)               52480     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0     

In [11]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG132.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [12]:
NNmodel2.compile(tf.keras.optimizers.Adam(),loss=mse)

In [14]:
Nep = 20
NNmodel2.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/20
18336/18336 [==============================] - 324s 18ms/step - loss: 41.6340 - val_loss: 35.3406
Epoch 2/20
18336/18336 [==============================] - 234s 13ms/step - loss: 32.7102 - val_loss: 33.4377
Epoch 3/20
18336/18336 [==============================] - 235s 13ms/step - loss: 31.3168 - val_loss: 32.3228
Epoch 4/20
18336/18336 [==============================] - 240s 13ms/step - loss: 30.5962 - val_loss: 30.9279
Epoch 5/20
18336/18336 [==============================] - 239s 13ms/step - loss: 30.1226 - val_loss: 30.9347
Epoch 6/20
18336/18336 [==============================] - 235s 13ms/step - loss: 29.7778 - val_loss: 30.4701
Epoch 7/20
18336/18336 [==============================] - 237s 13ms/step - loss: 29.4979 - val_loss: 30.9430
Epoch 8/20
18336/18336 [==============================] - 237s 13ms/step - loss: 29.2758 - val_loss: 29.9549
Epoch 9/20
18336/18336 [==============================] - 237s 13ms/step - loss: 29.0793 - val_loss: 30.0082
Epoch 10/20
18336/1

# 3) Repeat with only 30 levels

## 3.1) Preprocessing

In [ ]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/134_POG_RHTepsinput_dQdtdTdtepsotuput_test30lev_June.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages